# Геометрические преобразования и гомография

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os

In [ ]:
# Создаём тестовое изображение
def create_test_image():
    """Создаёт тестовое изображение с сеткой и объектами"""

    img = np.ones((400, 500, 3), dtype=np.uint8) * 240

    # Рисуем сетку
    for i in range(0, 500, 50):
        cv2.line(img, (i, 0), (i, 400), (200, 200, 200), 1)
    for i in range(0, 400, 50):
        cv2.line(img, (0, i), (500, i), (200, 200, 200), 1)

    # Добавляем объекты
    cv2.rectangle(img, (100, 100), (200, 200), (100, 150, 200), -1)
    cv2.circle(img, (350, 150), 50, (200, 100, 100), -1)

    # Треугольник
    triangle = np.array([[250, 250], [350, 250], [300, 320]], np.int32)
    cv2.fillPoly(img, [triangle], (150, 200, 150))

    # Текст
    cv2.putText(img, 'TRANSFORM', (150, 370),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (80, 80, 80), 2)

    return img

img_original = create_test_image()
img_rgb = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Исходное изображение')
plt.axis('off')
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Базовые геометрические преобразования
# ===============================

def create_euclidean_transform(tx, ty, theta):
    """Создаёт евклидово преобразование (перенос + поворот)"""
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)

    M = np.array([
        [cos_t, -sin_t, tx],
        [sin_t,  cos_t, ty],
        [0,      0,      1]
    ], dtype=np.float32)

    return M

def create_similarity_transform(tx, ty, theta, scale):
    """Создаёт преобразование подобия (перенос + поворот + масштаб)"""
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)

    M = np.array([
        [scale * cos_t, -scale * sin_t, tx],
        [scale * sin_t,  scale * cos_t, ty],
        [0,              0,              1]
    ], dtype=np.float32)

    return M

def create_affine_transform(tx, ty, angle, scale_x, scale_y, shear):
    """Создаёт аффинное преобразование"""
    cos_t = np.cos(angle)
    sin_t = np.sin(angle)

    # Матрица поворота
    R = np.array([
        [cos_t, -sin_t],
        [sin_t,  cos_t]
    ])

    # Матрица масштаба и сдвига
    S = np.array([
        [scale_x, shear],
        [0,       scale_y]
    ])

    # Комбинируем
    A = R @ S

    M = np.array([
        [A[0, 0], A[0, 1], tx],
        [A[1, 0], A[1, 1], ty],
        [0,       0,       1]
    ], dtype=np.float32)

    return M

# Создаём различные преобразования
transformations = {}

# Евклидово (перенос + поворот 30°)
transformations['Euclidean'] = create_euclidean_transform(50, 30, np.pi/6)

# Подобие (перенос + поворот + масштаб 0.8)
transformations['Similarity'] = create_similarity_transform(50, 30, np.pi/6, 0.8)

# Аффинное (полное)
transformations['Affine'] = create_affine_transform(50, 30, np.pi/6, 1.2, 0.9, 0.2)

# Проективное (гомография) - создаём позже

# Применяем преобразования
height, width = img_original.shape[:2]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Исходное изображение
axes[0].imshow(img_rgb)
axes[0].set_title('Исходное изображение')
axes[0].axis('off')
axes[0].grid(True, alpha=0.3)

for i, (name, M) in enumerate(transformations.items(), 1):
    # Применяем преобразование (используем первые 2 строки для cv2.warpAffine)
    M_affine = M[:2, :]
    transformed = cv2.warpAffine(img_original, M_affine, (width, height))
    transformed_rgb = cv2.cvtColor(transformed, cv2.COLOR_BGR2RGB)

    axes[i].imshow(transformed_rgb)
    axes[i].set_title(f'{name} Transform')
    axes[i].axis('off')
    axes[i].grid(True, alpha=0.3)

    print(f"\n{name} Transform:")
    print(M)

plt.tight_layout()
plt.show()

# Демонстрация композиции преобразований
print("\n🔄 Композиция преобразований:")

# Перенос → Поворот → Масштаб
T1 = create_euclidean_transform(100, 50, 0)  # Перенос
T2 = create_euclidean_transform(0, 0, np.pi/4)  # Поворот 45°
T3 = create_similarity_transform(0, 0, 0, 1.5)  # Масштаб 1.5

# Композиция (справа налево: сначала T1, потом T2, потом T3)
M_composed = T3 @ T2 @ T1

print("Композиция: Масштаб(Поворот(Перенос(x)))")
print(M_composed)

img_composed = cv2.warpAffine(img_original, M_composed[:2, :], (width * 2, height * 2))

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_composed, cv2.COLOR_BGR2RGB))
plt.title('Результат композиции преобразований')
plt.axis('off')
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Вычисление гомографии
# ===============================

# Создаём изображение с чёткими углами (документ)
def create_document_image():
    """Создаёт изображение документа"""
    img = np.ones((400, 300, 3), dtype=np.uint8) * 255

    # Рамка
    cv2.rectangle(img, (20, 20), (280, 380), (0, 0, 0), 2)

    # Заголовок
    cv2.putText(img, 'DOCUMENT', (60, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Линии текста
    for i in range(5):
        y = 120 + i * 40
        cv2.line(img, (40, y), (260, y), (150, 150, 150), 2)

    return img

doc_img = create_document_image()

# Определяем 4 угла исходного документа
src_points = np.float32([
    [20, 20],    # Верхний левый
    [280, 20],   # Верхний правый
    [280, 380],  # Нижний правый
    [20, 380]    # Нижний левый
])

# Определяем целевые позиции (с перспективой)
dst_points = np.float32([
    [50, 100],   # Верхний левый (сдвинут)
    [350, 50],   # Верхний правый
    [320, 400],  # Нижний правый
    [80, 450]    # Нижний левый
])

# Вычисляем гомографию с помощью OpenCV
H_cv, _ = cv2.findHomography(src_points, dst_points)

print("Гомография (OpenCV):")
print(H_cv)

# Применяем гомографию
transformed_doc = cv2.warpPerspective(doc_img, H_cv, (500, 600))

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Исходный документ с отмеченными точками
doc_with_points = doc_img.copy()
for i, pt in enumerate(src_points):
    cv2.circle(doc_with_points, tuple(pt.astype(int)), 5, (255, 0, 0), -1)
    cv2.putText(doc_with_points, str(i+1), tuple((pt + 10).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

axes[0].imshow(cv2.cvtColor(doc_with_points, cv2.COLOR_BGR2RGB))
axes[0].set_title('Исходный документ\n(синие точки = src_points)')
axes[0].axis('off')

# Трансформированный документ с отмеченными точками
transformed_with_points = transformed_doc.copy()
for i, pt in enumerate(dst_points):
    cv2.circle(transformed_with_points, tuple(pt.astype(int)), 5, (0, 255, 0), -1)
    cv2.putText(transformed_with_points, str(i+1), tuple((pt + 10).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

axes[1].imshow(cv2.cvtColor(transformed_with_points, cv2.COLOR_BGR2RGB))
axes[1].set_title('После гомографии\n(зелёные точки = dst_points)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Реализация DLT алгоритма
# ===============================

def compute_homography_dlt(src_pts, dst_pts):
    """
    Вычисляет гомографию используя DLT алгоритм

    Args:
        src_pts: исходные точки (Nx2)
        dst_pts: целевые точки (Nx2)

    Returns:
        H: матрица гомографии 3x3
    """

    n_points = len(src_pts)

    # Строим матрицу A (2n x 9)
    A = []

    for i in range(n_points):
        x, y = src_pts[i]
        x_prime, y_prime = dst_pts[i]

        # Два уравнения для каждой пары точек
        A.append([-x, -y, -1, 0, 0, 0, x*x_prime, y*x_prime, x_prime])
        A.append([0, 0, 0, -x, -y, -1, x*y_prime, y*y_prime, y_prime])

    A = np.array(A)

    # Решаем A * h = 0 используя SVD
    U, S, Vt = np.linalg.svd(A)

    # Решение = последний столбец V (последняя строка Vt)
    h = Vt[-1, :]

    # Преобразуем в матрицу 3x3
    H = h.reshape(3, 3)

    # Нормализуем (h33 = 1)
    H = H / H[2, 2]

    return H

# Вычисляем гомографию нашим методом
H_dlt = compute_homography_dlt(src_points, dst_points)

print("\nГомография (наша реализация DLT):")
print(H_dlt)

print("\nСравнение с OpenCV:")
print(f"Разность матриц: {np.max(np.abs(H_cv - H_dlt)):.6f}")

# Применяем нашу гомографию
transformed_dlt = cv2.warpPerspective(doc_img, H_dlt, (500, 600))

# Сравнение результатов
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(cv2.cvtColor(doc_img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(transformed_doc, cv2.COLOR_BGR2RGB))
axes[1].set_title('OpenCV findHomography()')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(transformed_dlt, cv2.COLOR_BGR2RGB))
axes[2].set_title('Наша реализация DLT')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Проверка ошибки репроекции
def reprojection_error(H, src_pts, dst_pts):
    """Вычисляет ошибку репроекции"""

    # Преобразуем src_pts в гомогенные координаты
    src_h = np.hstack([src_pts, np.ones((len(src_pts), 1))])

    # Применяем гомографию
    dst_proj_h = (H @ src_h.T).T

    # Преобразуем обратно в декартовы координаты
    dst_proj = dst_proj_h[:, :2] / dst_proj_h[:, 2:]

    # Вычисляем евклидово расстояние
    errors = np.linalg.norm(dst_pts - dst_proj, axis=1)

    return errors

errors_cv = reprojection_error(H_cv, src_points, dst_points)
errors_dlt = reprojection_error(H_dlt, src_points, dst_points)

print("\n📊 Ошибки репроекции:")
print(f"OpenCV: {errors_cv}")
print(f"DLT:    {errors_dlt}")
print(f"Средняя ошибка OpenCV: {np.mean(errors_cv):.6f} пикселей")
print(f"Средняя ошибка DLT:    {np.mean(errors_dlt):.6f} пикселей")

In [ ]:
# ===============================
# ЗАДАНИЕ 4: RANSAC для фильтрации выбросов
# ===============================

def ransac_homography(src_pts, dst_pts, threshold=5.0, n_iterations=1000, confidence=0.99):
    """
    RANSAC для вычисления гомографии

    Args:
        src_pts: исходные точки (Nx2)
        dst_pts: целевые точки (Nx2)
        threshold: порог для inliers (пиксели)
        n_iterations: количество итераций
        confidence: требуемая уверенность

    Returns:
        H: лучшая гомография
        inliers: маска inliers
    """

    n_points = len(src_pts)
    best_H = None
    best_inliers = []

    print(f"🔄 Выполняем RANSAC: {n_iterations} итераций...")

    for iteration in range(n_iterations):
        # 1. Случайно выбираем 4 точки
        indices = np.random.choice(n_points, 4, replace=False)

        src_sample = src_pts[indices]
        dst_sample = dst_pts[indices]

        # 2. Вычисляем гомографию по этим 4 точкам
        try:
            H = compute_homography_dlt(src_sample, dst_sample)
        except:
            continue

        # 3. Вычисляем ошибки для всех точек
        errors = reprojection_error(H, src_pts, dst_pts)

        # 4. Определяем inliers
        inliers = errors < threshold
        n_inliers = np.sum(inliers)

        # 5. Обновляем лучшую модель
        if n_inliers > len(best_inliers):
            best_inliers = inliers
            best_H = H

            if iteration % 100 == 0:
                print(f"   Итерация {iteration}: {n_inliers} inliers")

    # 6. Пересчитываем гомографию по всем inliers
    if np.sum(best_inliers) >= 4:
        best_H = compute_homography_dlt(src_pts[best_inliers], dst_pts[best_inliers])

    print(f"✅ RANSAC завершён: {np.sum(best_inliers)}/{n_points} inliers")

    return best_H, best_inliers

# Создаём данные с выбросами
n_good = 20
n_outliers = 5

# Генерируем "хорошие" точки
np.random.seed(42)
src_good = np.random.rand(n_good, 2) * 200 + 50
H_true = compute_homography_dlt(
    src_points,
    dst_points
)
dst_good = cv2.perspectiveTransform(src_good.reshape(-1, 1, 2), H_true).reshape(-1, 2)

# Добавляем шум к хорошим точкам
dst_good += np.random.randn(n_good, 2) * 2

# Генерируем выбросы (случайные точки)
src_outliers = np.random.rand(n_outliers, 2) * 200 + 50
dst_outliers = np.random.rand(n_outliers, 2) * 300 + 100

# Комбинируем
src_all = np.vstack([src_good, src_outliers])
dst_all = np.vstack([dst_good, dst_outliers])

# Вычисляем гомографию БЕЗ RANSAC
H_no_ransac = compute_homography_dlt(src_all, dst_all)

# Вычисляем гомографию С RANSAC
H_ransac, inliers_mask = ransac_homography(src_all, dst_all, threshold=5.0, n_iterations=500)

# Сравниваем результаты
print("\n📊 Сравнение с/без RANSAC:")

errors_no_ransac = reprojection_error(H_no_ransac, src_all, dst_all)
errors_ransac = reprojection_error(H_ransac, src_all, dst_all)

print(f"Средняя ошибка БЕЗ RANSAC: {np.mean(errors_no_ransac):.2f} пикселей")
print(f"Средняя ошибка С RANSAC:   {np.mean(errors_ransac):.2f} пикселей")

# Визуализация inliers и outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Без RANSAC
axes[0].scatter(src_all[:, 0], src_all[:, 1], c='blue', s=50, label='Source points')
axes[0].scatter(dst_all[:, 0], dst_all[:, 1], c='red', s=50, label='Destination points')
axes[0].set_title('Все точки (без фильтрации)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# С RANSAC
colors = ['green' if inlier else 'red' for inlier in inliers_mask]
axes[1].scatter(src_all[:, 0], src_all[:, 1], c='blue', s=50, alpha=0.5, label='Source')
axes[1].scatter(dst_all[:, 0], dst_all[:, 1], c=colors, s=50, label='Dest (green=inlier)')
axes[1].set_title(f'После RANSAC: {np.sum(inliers_mask)} inliers, {n_outliers} outliers')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Применяем обе гомографии к документу
doc_no_ransac = cv2.warpPerspective(doc_img, H_no_ransac, (500, 600))
doc_ransac = cv2.warpPerspective(doc_img, H_ransac, (500, 600))

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(cv2.cvtColor(doc_img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(doc_no_ransac, cv2.COLOR_BGR2RGB))
axes[1].set_title('БЕЗ RANSAC (с выбросами)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(doc_ransac, cv2.COLOR_BGR2RGB))
axes[2].set_title('С RANSAC (устойчиво к выбросам)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Практическое применение - коррекция перспективы
# ===============================

# Создаём изображение документа под углом
doc_perspective = create_document_image()

# Применяем перспективное искажение
src_corners = np.float32([[0, 0], [300, 0], [300, 400], [0, 400]])
dst_corners_persp = np.float32([[50, 80], [280, 20], [320, 370], [20, 420]])

H_distort = compute_homography_dlt(src_corners, dst_corners_persp)
doc_distorted = cv2.warpPerspective(doc_perspective, H_distort, (400, 500))

# Теперь корректируем (обратное преобразование)
H_correct = compute_homography_dlt(dst_corners_persp, src_corners)
doc_corrected = cv2.warpPerspective(doc_distorted, H_correct, (300, 400))

# Визуализация процесса
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(cv2.cvtColor(doc_perspective, cv2.COLOR_BGR2RGB))
axes[0].set_title('1. Исходный документ')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(doc_distorted, cv2.COLOR_BGR2RGB))
axes[1].set_title('2. С перспективным искажением\n(как фото под углом)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(doc_corrected, cv2.COLOR_BGR2RGB))
axes[2].set_title('3. После коррекции\n(выправлен)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# ДОМАШНЕЕ ЗАДАНИЕ

1. Реализуйте декомпозицию гомографии на R, t, n, d (поворот, перенос, нормаль плоскости, расстояние)

2. Создайте интерактивное приложение для коррекции перспективы:
- Пользователь кликает 4 угла документа
- Программа автоматически выправляет перспективу

3. Исследуйте влияние количества точек на точность:
- Протестируйте с 4, 8, 16, 32, 64 точками
- Постройте график ошибки vs количество точек
